torch.cuda.amp 为混合精度提供了方便的方法，其中一些操作使用 torch.float32（float）数据类型，而其他操作使用 torch.float16（half）。一些操作，如线性层和卷积，在 float16 中要快得多。其他操作，如减少，通常需要 float32 的动态范围。混合精度尝试将每个操作与其适当的数据类型匹配，这可以减少网络的运行时间和内存占用。

通常，“自动混合精度训练”将 torch.cuda.amp.autocast 和 torch.cuda.amp.GradScaler 一起使用。

此秘籍以默认精度测量简单网络的性能，然后逐步添加 autocast 和 GradScaler 以混合精度运行相同的网络并提高性能。

您可以下载并运行此配方作为独立的 Python 脚本。唯一的要求是 Pytorch 1.6+ 和支持 CUDA 的 GPU。

混合精度主要有利于启用 Tensor Core 的架构（Volta、Turing、Ampere）。这个秘籍应该在这些架构上显示出显着的 (2-3X) 加速。在较早的架构（Kepler、Maxwell、Pascal）上，您可能会观察到适度的加速。运行 nvidia-smi 以显示您的 GPU 架构。

In [1]:
import torch, time, gc

# Timing utilities
start_time = None

def start_timer():
    global start_time
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.synchronize()
    start_time = time.time()

def end_timer_and_print(local_msg):
    torch.cuda.synchronize()
    end_time = time.time()
    print("\n" + local_msg)
    print("Total execution time = {:.3f} sec".format(end_time - start_time))
    print("Max memory used by tensors = {} bytes".format(torch.cuda.max_memory_allocated()))

一个简单的网络

以下线性层和 ReLU 序列应该显示出混合精度的加速。

In [4]:
def make_model(in_size, out_size, num_layers):
    layers = []
    for _ in range(num_layers - 1):
        layers.append(torch.nn.Linear(in_size, in_size))
        layers.append(torch.nn.ReLU())
    layers.append(torch.nn.Linear(in_size, out_size))
    return torch.nn.Sequential(*tuple(layers)).cuda()

batch_size、in_size、out_size 和 num_layers 被选择得足够大，以使 GPU 充满工作。 通常，当 GPU 饱和时，混合精度提供最大的加速。 小型网络可能受 CPU 限制，在这种情况下，混合精度不会提高性能。 还选择了尺寸，使得线性层的参与维度是 8 的倍数，以允许在支持 Tensor Core 的 GPU 上使用 Tensor Core（请参阅下面的故障排除）。

练习：改变参与大小，看看混合精度加速如何变化。

In [5]:
batch_size = 512 # Try, for example, 128, 256, 513.
in_size = 4096
out_size = 4096
num_layers = 3
num_batches = 50
epochs = 3

# Creates data in default precision.
# The same data is used for both default and mixed precision trials below.
# You don't need to manually change inputs' dtype when enabling mixed precision.
data = [torch.randn(batch_size, in_size, device="cuda") for _ in range(num_batches)]
targets = [torch.randn(batch_size, out_size, device="cuda") for _ in range(num_batches)]

loss_fn = torch.nn.MSELoss().cuda()

# 默认精度

如果没有 torch.cuda.amp，以下简单网络以默认精度（torch.float32）执行所有操作：

In [6]:
net = make_model(in_size, out_size, num_layers)
opt = torch.optim.SGD(net.parameters(), lr=0.001)

start_timer()
for epoch in range(epochs):
    for input, target in zip(data, targets):
        output = net(input)
        loss = loss_fn(output, target)
        loss.backward()
        opt.step()
        opt.zero_grad() # set_to_none=True here can modestly improve performance
end_timer_and_print("Default precision:")

/data/miniconda3/envs/torch19/lib/python3.9/site-packages/torch/cuda/memory.py:260: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(



Default precision:
Total execution time = 2.155 sec
Max memory used by tensors = 1367458816 bytes


# 添加自动广播

torch.cuda.amp.autocast 的实例用作上下文管理器，允许您的脚本区域以混合精度运行。

在这些区域中，CUDA 操作以 autocast 选择的 dtype 运行，以在保持准确性的同时提高性能。 请参阅 Autocast Op Reference 以了解有关为每个 op 选择什么精度 autocast 以及在什么情况下的详细信息。

In [7]:
for epoch in range(0): # 0 epochs, this section is for illustration only
    for input, target in zip(data, targets):
        # Runs the forward pass under autocast.
        with torch.cuda.amp.autocast():
            output = net(input)
            # output is float16 because linear layers autocast to float16.
            assert output.dtype is torch.float16

            loss = loss_fn(output, target)
            # loss is float32 because mse_loss layers autocast to float32.
            assert loss.dtype is torch.float32

        # Exits autocast before backward().
        # Backward passes under autocast are not recommended.
        # Backward ops run in the same dtype autocast chose for corresponding forward ops.
        loss.backward()
        opt.step()
        opt.zero_grad() # set_to_none=True here can modestly improve performance

# 添加 GradScaler

在以混合精度训练时，梯度缩放有助于防止小幅度的梯度刷新为零（“下溢”）。

torch.cuda.amp.GradScaler 方便地执行梯度缩放的步骤。

In [8]:
# Constructs scaler once, at the beginning of the convergence run, using default args.
# If your network fails to converge with default GradScaler args, please file an issue.
# The same GradScaler instance should be used for the entire convergence run.
# If you perform multiple convergence runs in the same script, each run should use
# a dedicated fresh GradScaler instance.  GradScaler instances are lightweight.
scaler = torch.cuda.amp.GradScaler()

for epoch in range(0): # 0 epochs, this section is for illustration only
    for input, target in zip(data, targets):
        with torch.cuda.amp.autocast():
            output = net(input)
            loss = loss_fn(output, target)

        # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
        scaler.scale(loss).backward()

        # scaler.step() first unscales the gradients of the optimizer's assigned params.
        # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
        # otherwise, optimizer.step() is skipped.
        scaler.step(opt)

        # Updates the scale for next iteration.
        scaler.update()

        opt.zero_grad() # set_to_none=True here can modestly improve performance

# 所有在一起：“自动混合精度”

（下面还演示了启用，自动转换和 GradScaler 的可选便利参数。如果为 False，自动转换和 GradScaler 的调用将变为无操作。这允许在没有 if/else 语句的情况下在默认精度和混合精度之间切换。）

In [9]:
use_amp = True

net = make_model(in_size, out_size, num_layers)
opt = torch.optim.SGD(net.parameters(), lr=0.001)
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

start_timer()
for epoch in range(epochs):
    for input, target in zip(data, targets):
        with torch.cuda.amp.autocast(enabled=use_amp):
            output = net(input)
            loss = loss_fn(output, target)
        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()
        opt.zero_grad() # set_to_none=True here can modestly improve performance
end_timer_and_print("Mixed precision:")


Mixed precision:
Total execution time = 2.056 sec
Max memory used by tensors = 1585621504 bytes


# 检查/修改渐变（例如，剪辑）

scaler.scale(loss).backward() 产生的所有梯度都被缩放。 如果你想修改或检查 backward() 和 scaler.step(optimizer) 之间参数的 .grad 属性，你应该首先使用 scaler.unscale_(optimizer) 取消缩放它们。

In [10]:
for epoch in range(0): # 0 epochs, this section is for illustration only
    for input, target in zip(data, targets):
        with torch.cuda.amp.autocast():
            output = net(input)
            loss = loss_fn(output, target)
        scaler.scale(loss).backward()

        # Unscales the gradients of optimizer's assigned params in-place
        scaler.unscale_(opt)

        # Since the gradients of optimizer's assigned params are now unscaled, clips as usual.
        # You may use the same value for max_norm here as you would without gradient scaling.
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=0.1)

        scaler.step(opt)
        scaler.update()
        opt.zero_grad() # set_to_none=True here can modestly improve performance

# 保存/恢复

要以按位精度保存/恢复启用 Amp 的运行，请使用 scaler.state_dict 和 scaler.load_state_dict。

保存时，将缩放器状态字典与通常的模型和优化器状态字典一起保存。 在任何前向传递之前的迭代开始时或在 scaler.update() 之后的迭代结束时执行此操作。

In [11]:
checkpoint = {"model": net.state_dict(),
              "optimizer": opt.state_dict(),
              "scaler": scaler.state_dict()}
# Write checkpoint as desired, e.g.,
# torch.save(checkpoint, "filename")

恢复时，将缩放器状态字典与模型和优化器状态字典一起加载。

In [12]:
# Read checkpoint as desired, e.g.,
# dev = torch.cuda.current_device()
# checkpoint = torch.load("filename",
#                         map_location = lambda storage, loc: storage.cuda(dev))
net.load_state_dict(checkpoint["model"])
opt.load_state_dict(checkpoint["optimizer"])
scaler.load_state_dict(checkpoint["scaler"])

如果检查点是从没有 Amp 的运行中创建的，并且您想继续使用 Amp 进行训练，请照常从检查点加载模型和优化器状态。 检查点不会包含保存的缩放器状态，因此请使用 GradScaler 的新实例。

如果检查点是从使用 Amp 的运行中创建的，并且您想在没有 Amp 的情况下继续训练，请照常从检查点加载模型和优化器状态，并忽略保存的缩放器状态。

# 推理/评估

autocast 可以单独使用来包装推理或评估前向传递。 GradScaler 不是必需的。

进阶课题

有关高级用例，请参阅自动混合精度示例，包括：
* 梯度积累
* 梯度惩罚/向后加倍
* 具有多个模型、优化器或损失的网络
* 多个 GPU（torch.nn.DataParallel 或 torch.nn.parallel.DistributedDataParallel）
* 自定义 autograd 函数（torch.autograd.Function 的子类）

如果您在同一个脚本中执行多次收敛运行，则每次运行都应使用专用的全新 GradScaler 实例。 GradScaler 实例是轻量级的。

如果您正在向调度程序注册自定义 C++ 操作，请参阅调度程序教程的自动广播部分。

# 故障排除

## Amp 的加速无关紧要

您的网络可能无法通过工作使 GPU 饱和，因此受 CPU 限制。 Amp 对 GPU 性能的影响无关紧要。
使 GPU 饱和的粗略经验法则是在不运行 OOM 的情况下尽可能多地增加批处理和/或网络大小。
尽量避免过度的 CPU-GPU 同步（.item() 调用，或从 CUDA 张量打印值）。
尽量避免多个小型 CUDA 操作的序列（如果可以，将它们合并为几个大型 CUDA 操作）。
您的网络可能受 GPU 计算限制（大量 matmul/卷积），但您的 GPU 没有 Tensor Core。 在这种情况下，预计会降低加速比。
Matmul 维度不是 Tensor Core 友好的。 确保 matmuls 的参与大小是 8 的倍数。（对于带有编码器/解码器的 NLP 模型，这可能很微妙。此外，卷积曾经对 Tensor Core 使用具有类似的大小限制，但对于 CuDNN 7.3 及更高版本，没有这样的限制 存在。请参阅此处获取指导。）

## 损失是 inf/NaN

首先，检查您的网络是否适合高级用例。 另请参阅比 binary_cross_entropy 更喜欢 binary_cross_entropy_with_logits。

如果您确信自己的 Amp 使用是正确的，则可能需要提交问题，但在此之前，收集以下信息会很有帮助：

单独禁用 autocast 或 GradScaler（通过将 enabled=False 传递给它们的构造函数）并查看 infs/NaN 是否持续存在。
如果您怀疑网络的一部分（例如，复杂的损失函数）溢出，请在 float32 中运行该前向区域并查看 infs/NaN 是否仍然存在。 autocast 文档字符串的最后一个代码片段显示强制子区域在 float32 中运行（通过本地禁用自动广播并强制转换子区域的输入）。

## 类型不匹配错误（可能表现为 CUDNN_STATUS_BAD_PARAM）

Autocast 试图涵盖所有受益于或需要转换的操作。 接受明确覆盖的操作是根据数值属性选择的，但也根据经验。 如果您在启用自动广播的前向区域或后向传递区域中看到类型不匹配错误，则可能是自动广播错过了操作。

请提交有关错误回溯的问题。 在运行脚本之前导出 TORCH_SHOW_CPP_STACKTRACES=1 以提供有关哪个后端操作失败的细粒度信息。